<a href="https://colab.research.google.com/github/Yurovskyy/TP3/blob/main/TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
import plotly.express as px

In [2]:
# @title Carregando o dataset

# URL do arquivo CSV
url = 'https://github.com/Yurovskyy/TP3/raw/main/output.csv'

# Baixar o arquivo CSV da web
response = requests.get(url)

# Verificar se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Ler o conteúdo do arquivo CSV usando o Pandas
    df = pd.read_csv(url)

# Se a solicitação não for bem-sucedida
else:
    print('Falha ao baixar o arquivo CSV.')


In [3]:
# @title Exploração inicial do dataset

dimensoes = df.shape
print("Número de entradas(linhas):", dimensoes[0])
print("Número de colunas(variáveis):", dimensoes[1])

# identificar o tipo de cada variável
display(df.dtypes)

# ver as 5 primeiras entradas do conjunto de treino
display(df.head())

Número de entradas(linhas): 4944
Número de colunas(variáveis): 14


,0
Index,float64
firm,int64
BE,float64
ME,float64
ME.lag1,float64
r_i,float64
r.3.0,float64
r.12.4,float64
AC,float64
CA,float64


,Index,firm,BE,ME,ME.lag1,r_i,r.3.0,r.12.4,AC,CA,DR,OFC,ECI,industry
0,2020.75,1,1.179763e+09,1.217182e+09,1.200622e+09,0.089471,-0.013699,-0.618101,0.500000,0.694444,0.750000,0.433333,0.805556,Outros
1,2020.50,1,1.262906e+09,1.064582e+09,1.217182e+09,-0.013699,0.142185,-0.471392,0.500000,0.611111,0.750000,0.400000,0.833333,Outros
2,2022.00,1,1.205650e+09,1.354615e+09,1.761918e+09,0.181542,0.259363,0.039740,0.527778,0.722222,0.750000,0.666667,0.833333,Outros
3,2021.25,1,1.190537e+09,1.312840e+09,1.064523e+09,0.267479,-0.209721,0.217958,0.500000,0.694444,0.750000,0.433333,0.805556,Outros
4,2023.00,1,1.072554e+09,2.471660e+09,2.544702e+09,0.042154,0.029124,0.337521,0.527778,0.888889,0.958333,0.766667,0.722222,Outros


In [4]:
# @title Valores faltantes
# @markdown Como eu disse, não temos valores faltantes!

# ver a porcentagem valores faltantes
display((df.isnull().sum() / df.shape[0]).sort_values(ascending=False))

,0
Index,0.0
firm,0.0
BE,0.0
ME,0.0
ME.lag1,0.0
r_i,0.0
r.3.0,0.0
r.12.4,0.0
AC,0.0
CA,0.0


In [5]:
# @title Descrição dos valores categóricos

display(df.describe(include=['O']))

,industry
count,4944
unique,20
top,Outros
freq,1145
